# Handling SC2017 PRP productions

This module has functions to:
- create all the workload
- move from state to state [created,training,trained,generating,generated,transporting,done]


In [ ]:
from time import time
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
index_name='sc2017'

create template

In [ ]:
! curl -XPOST "atlas-kibana.mwt2.org:9200/_template/sc2017" -d @SC2017_template.json

 [--nb-epochs NB_EPOCHS] 
 [--batch-size BATCH_SIZE]
 [--latent-size LATENT_SIZE] 
 [--disc-lr DISC_LR]
 [--gen-lr GEN_LR] 
 [--adam-beta ADAM_BETA] 
 [--prog-bar]
 [--no-attn] 
 [--debug] 
 [--d-pfx D_PFX] 
 [--g-pfx G_PFX]
 dataset

In [ ]:
def create_workload():
    
    # clean all
    try:
        es.indices.delete(index='sc2017')
    except:
        print("not there?")
    
    id=0
    
    l0 = ['--nb-epochs=50','--nb-epochs=100','--nb-epochs=200',]
    l1 = ['--disc-lr=0.0001','--disc-lr=0.0002','--disc-lr=0.0005']
    l2  = ['--gen-lr=0.00001','--gen-lr=0.00002','--gen-lr=0.00005']
    l3    = ['', '--no-attn']
    l4    = ['gamma.yaml', 'eplus.yaml', 'pion.yaml']
    for a in l0:
        for b in l1:
            for c in l2:
                for d in l3:
                    for e in l4:
                        doc={}
                        doc['created']=int(time()*1000)
                        doc['status']='created'
                        doc['training_options']=['--output_folder=/data/CaloGAN/weigths/'+str(id), a, b, c, d, e]
                        doc['generating_options']=[
                                                   '--input_folder=/data/CaloGAN/weigths/'+str(id), 
                                                   '--output_folder=/data/CaloGAN/outputs/'+str(id), 
                                                   '--sets=10', 
                                                   '--showers=100000'
                                                  ]
                        id+=1
#                         print(doc)
                        es.create(index=index_name, doc_type='doc', id=id, body=doc)

In [ ]:
def get_training_job():
    my_query={
        "size": 1,
        "query":{ 
            "term": {"status":"created"} 
        }
    }

    res = es.search(index=index_name, body=my_query )
    res=res['hits']
    if res['total']==0:
        print('no training jobs at the moment.')
        return
    res_id=res['hits'][0]['_id']
    res=res['hits'][0]['_source']
    print(res)
        
    #now update to training...
    
    res['status']='training'
    res['start_training']=int(time()*1000)
    es.update(index=index_name, doc_type='doc', id=res_id, body={"doc":res})
    return (res_id, res)

In [ ]:
def done_training(id):
    res={'status':'trained', 'end_training':int(time()*1000) }
    es.update(index=index_name, doc_type='doc', id=id, body={"doc":res})

In [ ]:
def get_generating_job():
    my_query={
        "size": 1,
        "query":{ 
            "term": {"status":"trained"} 
        }
    }

    res = es.search(index=index_name, body=my_query )
    res=res['hits']
    if res['total']==0:
        print('no generating jobs at the moment.')
        return
    res_id=res['hits'][0]['_id']
    res=res['hits'][0]['_source']
    print(res)
        
    #now update to generating...
    
    res['status']='generating'
    res['start_generating']=int(time()*1000)
    es.update(index=index_name, doc_type='doc', id=res_id, body={"doc":res})
    return (res_id, res)

In [ ]:
def done_generating(id):
    res={'status':'generated', 'end_generating':int(time()*1000) }
    es.update(index=index_name, doc_type='doc', id=id, body={"doc":res})

In [ ]:
def get_transfering_job():
    my_query={
        "size": 1,
        "query":{ 
            "term": {"status":"generated"} 
        }
    }

    res = es.search(index=index_name, body=my_query )
    res=res['hits']
    if res['total']==0:
        print('no transfering jobs at the moment.')
        return
    res_id=res['hits'][0]['_id']
    res=res['hits'][0]['_source']
    print(res)
        
    #now update to transfering...
    
    res['status']='transfering'
    res['start_generating']=int(time()*1000)
    es.update(index=index_name, doc_type='doc', id=res_id, body={"doc":res})
    return (res_id, res)

In [ ]:
def done_transfering(id):
    res={'status':'transfered', 'end_transfering':int(time()*1000) }
    es.update(index=index_name, doc_type='doc', id=id, body={"doc":res})

Test all the steps

In [ ]:
create_workload()
(id, job) = get_training_job()
print(id, job)
sleep(5)
done_training(id)

(id, job) = get_generating_job()
print(id, job)
sleep(5)
done_generating(id)

(id, job) = get_transfering_job()
print(id, job)
sleep(5)
done_transfering(id)